In [11]:
from pathlib import Path
import pandas as pd
import json
import numpy as np
import spacy
# import spacy_cleaner
# from spacy_cleaner.processing import removers, replacers, mutators

import time


# from src.utils.logging_utils import create_logger

In [13]:
parent_folder_path = Path().cwd().parent
prep_data_path = parent_folder_path / 'data' / 'job_posts'/ 'prep_data'
model_path = parent_folder_path / 'data' / 'job_posts'/ 'artifacts' / 'gpt_models'
job_descr_path = parent_folder_path / 'data' /'job_posts'/ 'original_data/jd1.json'

## chatgpt vs gpt4all constants
# gpt_output_path = prep_data_path / 'chatgpt_output.csv'
# gpt_model = gpt_model_str = "gpt-3.5-turbo"
gpt_output_path = prep_data_path /'gpt4all_output.csv'
gpt_model_str = 'ggml-gpt4all-j-v1.3-groovy'




In [14]:
# question_string = 'I am looking for a job. List the skills you need to apply to this job posting: '
# question_string = 'what do I need to know in order to apply to this job: '
# question_string = 'What qualifications do I need to have in order to apply to this job: '

# question_string = 'List the hard skills needed to apply to the job posting above as bulletpoints'
# question_string = 'List the hard tech skills, responsibilities and technical tool knowledge the job posting above requires.'
question_string = ''


In [15]:
system_string = "Act like a researcher. Give me a list of keywords for a job description I will provide you with that I can use to find a candidate for the position. Act like an expert in this field, include additional keywords that may not be in the job description."
user_string = 'List the hard tech skills, responsibilities and technical tool knowledge the job posting above requires.'

In [16]:
class jobPostIingest:
    """
    Ingestion of job posts, feeding in only the ones that haven't yet been processed.
    """

    def __init__(self, job_descr_path: str, gpt_output_path: str):
        self.job_descr_path = job_descr_path
        self.gpt_output_path = gpt_output_path

    def ingest_jp(self):
        """
        Reads job descriptions from a JSON file at `job_descr_path` and appends any new responses generated by ChatGPT to the end of the file. 
        Removes duplicate rows based on all columns except 'salaries' and 'skill_summary'.

        
        Args:
            job_descr_path (str): Path to JSON file containing job descriptions.
            gpt_output_path (str): Path to CSV file containing ChatGPT responses.
        
        Returns:
            pandas.DataFrame: A merged DataFrame containing unique rows of job descriptions and ChatGPT responses.
        """
        
        df = pd.read_json(job_descr_path)
        column_mask = (df.columns != 'salaries') & (df.columns != f'skill_summary')
        df = df.drop_duplicates(subset = df.columns[column_mask])
        if gpt_output_path.exists():
            df_gpt = pd.read_csv(gpt_output_path)
            df = pd.concat([df,df_gpt]).drop_duplicates(subset = df.columns[column_mask], keep = False)
        return df

In [18]:
import gpt4all
from pathlib import Path
from typing import List, Dict
import openai
from dotenv import load_dotenv
import os
import time

from abc import ABC, abstractclassmethod

class abstractGptModel(ABC):
    
    @abstractclassmethod
    def gpt_prompt_return():
        pass

class gpt4All(abstractGptModel):
    """Wrapper class for the GPT-4 All model from gpt4all.

    Attributes:
        gpt_model_str (str): The name of the GPT-4 All model to use.
        model_path (Path): The path to the directory containing the model files.

    Methods:
        gpt_prompt_return(message_dict: List[Dict[str, str]]) -> str:
            Given a list of message dictionaries, returns the completion generated by the GPT-4 All model.
            Each message dictionary should have a 'speaker' key indicating which speaker the message is from,
            and a 'text' key containing the text of the message.
    """

    def __init__(self, gpt_model_str: str, model_path: Path) -> None:
        """Creates a new gpt4All instance.

        Args:
            gpt_model_str (str): The name of the GPT-4 All model to use.
            model_path (Path): The path to the directory containing the model files.
        """
        self.gpt_model_str = gpt_model_str
        self.model_path = model_path

    def gpt_prompt_return(self, prompt: List[Dict[str, str]]) -> str:
        """Given a list of message dictionaries, returns the completion generated by the GPT-4 All model.

        Args:
            message_dict (List[Dict[str, str]]): A list of message dictionaries.
                Each dictionary should have a 'speaker' key indicating which speaker the message is from,
                and a 'text' key containing the text of the message.

        Returns:
            str: The completion generated by the GPT-4 All model.
        """
        gpt_model = gpt4all.GPT4All(self.gpt_model_str, model_path=str(self.model_path))
        out = gpt_model.chat_completion(default_prompt_footer=False,
                                         default_prompt_header=False,
                                         messages=prompt,
                                         verbose=False,
                                         streaming=False)
        return out
    



class chatGpt(abstractGptModel):
    """Wrapper class for the GPT model from chatgpt.

    Attributes:
        gpt_model_str (str): The name of the GPT model (3/3.5/4) to use.
        model_path (Path): The path to the directory containing the model files.

    Methods:
        gpt_prompt_return(message_dict: List[Dict[str, str]]) -> str:
            Given a list of message dictionaries, returns the completion generated by the GPT model.
            Each message dictionary should have a 'speaker' key indicating which speaker the message is from,
            and a 'text' key containing the text of the message.
    """

    def __init__(self, gpt_model_str: str, model_path: None) -> None:
        """Creates a new chatGpt instance.

        Args:
            gpt_model_str (str): The name of the chatgpt model (3/3.5/4) to use.
            model_path (Path): The path to the directory containing the model files.
        """
        self.gpt_model_str = gpt_model_str

    @staticmethod
    def get_key_from_env() -> None:
        """
        Set api key from .env file. This file should contain an entry OPENAI_API_KEY = 'xxx'
        """
        load_dotenv()
        openai.api_key = os.environ.get('OPENAI_API_KEY')


    def gpt_prompt_return(self, prompt: List[Dict[str, str]]) -> str:
        """Given a list of message dictionaries, returns the completion generated by the GPT-4 All model.

        Args:
            message_dict (List[Dict[str, str]]): A list of message dictionaries.
                Each dictionary should have a 'speaker' key indicating which speaker the message is from,
                and a 'text' key containing the text of the message.

        Returns:
            str: The completion generated by the chatgpt model.
        """
        try:
            self.get_key_from_env()  # set api key
            out = openai.ChatCompletion.create(
                model= gpt_model_str,
                messages=prompt
                )
            time.sleep(25)  # free account has throughput limitations. This spreads throughput out for free key
        except:
            time.sleep(60*30)   # after n amount of prompts, free account time out to give priority to paid accounts. This will wait untill free accounts can work again 
            out = openai.ChatCompletion.create(
                model= gpt_model_str,
                messages=prompt
                )
        
        return out

In [20]:
class gptPredict:
    """
    This class applies a GPT language model to generate text predictions based on a provided input and saves the 
    results to a CSV file. 

    Args:
        df (pd.DataFrame): A pandas DataFrame containing the data to be processed.
        model (abstractGptModel): An instance of an abstractGptModel object representing the GPT language model to be used.
        system_string (str): A string message from the system that will introduce each text prediction prompt.
        user_string (str): A string message from the user that will be appended to the end of each text prediction prompt.
        gpt_output_path (Path): A pathlib.Path object representing the path where the generated text predictions 
                                should be saved as a CSV file.

    Attributes:
        model (abstractGptModel): The instance of the abstractGptModel object representing the GPT language model being used.
        system_string (str): A string message from the system that introduces each text prediction prompt.
        user_string (str): A string message from the user that is appended to the end of each text prediction prompt.
        gpt_output_path (Path): A pathlib.Path object representing the path where the generated text predictions 
                                should be saved as a CSV file.
        df (pd.DataFrame): The pandas DataFrame containing the data to be processed.
        prompt (List[Dict[str, str]]): A list of message dictionaries with a user message and speaker role for use in generating text predictions.

    Methods:
        message_dict(self, description) -> List[Dict[str, str]]:
            Build chatgpt-acceptable dictionary input with a user message.

        gpt_out_return(self, description: str) -> str:
            Generate a text prediction using the GPT language model for a given prompt string. Returns the generated text prediction.

        gpt_prompt(self, df_row: pd.Series) -> str:
            Generates a text prediction using the GPT language model for a given row of data in the pandas DataFrame. Returns the generated text prediction.

        gpt_prompt_save_csv(self, df_row: pd.Series) -> None:
            Generates a text prediction using the GPT language model for a given row of data in the pandas DataFrame and saves the result to the specified CSV file.

        apply_lambda_save_csv(self):
            Applies the gpt_prompt_save_csv method to each row of data in the pandas DataFrame and saves the resulting text predictions to the specified CSV file.
    """


    def __init__(self, df: pd.DataFrame, model: abstractGptModel, system_string: str, user_string: str, gpt_output_path: Path) -> None:
        """
        Initializes an instance of the gptPredict class.

        Args:
            df (pd.DataFrame): A pandas DataFrame containing job descriptions to analyze.
            model (abstractGptModel): An instance of a class implementing the abstractGptModel interface.
            system_string (str): A string representing the system prompt when interacting with chatgpt model.
            user_string (str): A string representing the user input prompt for entering job description in chatgpt model.
            gpt_output_path (Path): A Path object representing the location where the gpt model generated output is to be saved.
        """

        self.model = model
        self.system_string = system_string
        self.user_string = user_string
        self.gpt_output_path = gpt_output_path
        self.df = df



    def message_dict(self, description) -> List[Dict[str, str]]:
        """
        Builds a chatgpt-compatible dictionary input with a user message.

        Args:
            description (str): A string representing a job description.

        Returns:
            List[Dict[str, str]]: A list of message dictionaries. Each dictionary should have a 'speaker' key indicating which speaker the message is from, and a 'text' key containing the text of the message.
        """


        # return [{"role": "system", "content": "Act like a researcher. Give me a list of keywords for a job description I will provide you with that I can use to find a candidate for the position. Act like an expert in this field, include additional keywords that may not be in the job description."},
        #         {"role": "user", "content": prompt_str(description, question_string)}]
        # return [{"role": "system", "content": "Act like a researcher. Give me a list of keywords for a job description I will provide you with that I can use to find a candidate for the position. Act like an expert in this field, include additional keywords that may not be in the job description."},
        #         {"role": "user", "content": description}] 
        self.prompt =  [{"role": "system", "content": self.system_string},
                        {"role": "user", "content": description + '. ' + self.user_string}] 
        return self



    def gpt_prompt(self, df_row: pd.Series) -> str:
        """
        Generates a GPT model completion for a given DataFrame row containing a job description.
        In case of an empty string, a period or None it returns an empty string

        Args:
            df_row (pd.Series): A series representing a single row of a pandas DataFrame containing job descriptions.

        Returns:
            str: The completion generated by the chatgpt model.
        """

        out = self.message_dict(df_row['description']).model.gpt_prompt_return(self.prompt)
        try:
            ret = str(out["choices"][0]["message"]['content'])
            if ret == '.' or ret == 'None':
                # can also try cleaning up with spacy
                ret = ''
            return ret
        except:
            return ''
   


    def gpt_prompt_save_csv(self, df_row: pd.Series) -> None:
        """
        Generate a GPT model completion for a given DataFrame row containing a job description and append the result to a specified CSV file.

        Args:
            df_row (pd.Series): A series representing a single row of a pandas DataFrame containing job descriptions.
        """

        df_row[f'hard_skills_{gpt_model_str}'] = self.gpt_prompt(df_row)

        header_t_or_f = False if self.gpt_output_path.exists() else True 
        df_row.to_frame().T.to_csv(path_or_buf=self.gpt_output_path, 
                                    mode='a',
                                    header=header_t_or_f,
                                    index=False)


    def apply_lambda_save_csv(self) -> None:
        """
        Apply lambda function on each row of input DataFrame and save the corresponding GPT model completion as a CSV file.
        """

        ret = self.df.apply(lambda df_row: self.gpt_prompt_save_csv(df_row), axis=1)


In [21]:
df = jobPostIingest(job_descr_path, gpt_output_path).ingest_jp()

In [22]:
df = df.iloc[0:1,:]

In [23]:
description = df['description']

In [25]:
model = gpt4All(gpt_model_str, model_path)
gpt = gptPredict(df, model, system_string, user_string, gpt_output_path)
gpt.apply_lambda_save_csv()


Found model file at  d:\\\\project\\\\ai_job_autopilot\\\\gitlab\\\\ai_core\\\\data\\\\job_posts\\\\artifacts\\\\gpt_models\\ggml-gpt4all-j-v1.3-groovy.bin



In [ ]:
df

,title,companyName,location,description,salaries,hard_skills_ggml-gpt4all-j-v1.3-groovy
517,Data Scientist I,IGT,Canada,IGT (NYSE:IGT) is a global leader in gaming. W...,[{'link': 'https://www.glassdoor.ca/Salary/CIB...,NaN


'ai_core'